# Initialization & setup

In [0]:
!pip uninstall tensorflow; pip install tensorflow==2.0.0-beta1

Uninstalling tensorflow-2.0.0b1:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.0.0b1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? n


In [3]:
#from google.colab import drive

#root = "/content/gdrive"
#drive.mount(root)
import sys
ROOT = '/content/drive/'
PROJ = 'My Drive/Tesi/'
LOG = PROJ + 'logs/file.log'
PROJ_COMPLETE_PATH = ROOT + PROJ
LOG_COMPELTE_PATH = ROOT + LOG

if(PROJ_COMPLETE_PATH not in sys.path):
  sys.path.append(PROJ_COMPLETE_PATH)

sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/Tesi/']

In [0]:
### uncomment these line if there isn't already fuzzylearn repo in google drive
#!rm -rf {PROJ_COMPLETE_PATH}
#!mkdir {PROJ_COMPLETE_PATH}
#!git clone https://github.com/tomgeek27/fuzzylearn {PROJ_COMPLETE_PATH}

### uncomment this line if you need to update your repo (updated from develop branch)
#!cd {PROJ_COMPLETE_PATH}; git pull origin develop

In [0]:
import uuid

def log_init(filename="/content/drive/My Drive/Tesi/logs/file.log"):
  import logging

  logger = logging.getLogger(__name__)
  logger.setLevel(logging.INFO)
  if(not logger.hasHandlers()):
    f_handler = logging.FileHandler(filename)
    f_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

    c_handler = logging.StreamHandler()
    c_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(f_handler)
    logger.addHandler(c_handler)

  return logger

# Data processing

In [6]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

olivetti = fetch_olivetti_faces()

olivetti_data = olivetti.data
olivetti_labels = olivetti.target

olivetti_data_tsne = TSNE(n_components=2).fit_transform(olivetti_data)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(olivetti_data_tsne, olivetti_labels, stratify=olivetti_labels, random_state=0, test_size=.3)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [0]:
def get_classes(dataset_labels):
  return np.unique(dataset_labels)

def str2fuzzifier(f_str):
  if(f_str == 'LinearFuzzifier'):
    return fl.fuzzifiers.LinearFuzzifier
  elif(f_str == 'QuantileLinearPiecewiseFuzzifier'):
    return fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier

def str2kernel(k_str):
  if(k_str == 'GaussianKernel'):
    return fl.kernel.GaussianKernel

In [8]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedShuffleSplit
import fuzzylearn as fl
import logging
from sklearn.model_selection import GroupKFold
import importlib

importlib.reload(fl)
def estimate_membership_holdout(x, y, c=1, k=fl.kernel.GaussianKernel(sigma=.5), fuzzifier=fl.fuzzifiers.LinearFuzzifier, y_needs_mapping=False, label=""):
  if(y_needs_mapping):
    y = map_labels(y, label)
  model = fl.FuzzyInductor(c=c, k=k, fuzzifier=fuzzifier).fit(x, y)

  return model

def grid_search(x, y, params, logger=None, y_needs_mapping=False, label="", folds=3):
  check_label(label)
  
  if(y_needs_mapping):
    y = map_labels(y, label)

  log_info(logger, label, "Started GridSearchCV with: %s", params)
  
  clf = GridSearchCV(fl.FuzzyInductor(), params, cv=stratified_folds(x, y, folds=folds)).fit(x, y)

  log_info(logger, label, "CV_RESULTS_: %s", clf.cv_results_)
  log_info(logger, label, "BEST_SCORE_: %s", clf.best_score_)
  log_info(logger, label, "BEST_PARAMS_: %s", clf.best_params_)
  return clf


def nested_grid_search(x, y, params, logger=None, y_needs_mapping=False, label="", folds=3, innerFolds=3):
  check_label(label)

  log_info(logger, label, "Started Nested-GridSearchCV with: %s", params)
  if(y_needs_mapping):
    y = map_labels(y, label)

  clf = grid_search(x, y, params, logger=logger, y_needs_mapping=not y_needs_mapping, label=label, folds=innerFolds)
  results = cross_val_score(clf.best_estimator_, x, y, cv=stratified_folds(x, y, folds=folds), verbose=10)

  log_info(logger, label, "NESTED CROSS VALIDATION with %s fold: %s", folds, results)
  log_info(logger, label, "NESTED CROSS VALIDATION (mean): %s", np.mean(results))

def log_info(logger, label, message, *args):
  if(logger != None):
    logger.info("(" + str(label) + ") - " + message, *args)

def map_labels(y, label):
    assert(label != "")
    return [1 if elem == label else 0 for elem in y]

def check_label(label):
  assert(label != "" or label is not None) #is important to know which class is training 

def stratified_folds(X, y, random_state=0, folds=3):
  return StratifiedShuffleSplit(n_splits=folds, random_state=random_state).split(X, y)


gurobi not available


gurobi not available


```
estimate_membership_holdout
```

Questa funzione rappresenta è utile a definire una model_selection effettuata tramite un semplice holdout, la quale vede come argomenti tutti fgli argomenti del modello 'FuzzyInductor'

# Logging

Here you can find all tools that you need to retrieve information from _file.log_

In [10]:
def lastAnalyzedClass(lines):
  """Return the index of the last trained class, you could use it for train all model after the last trained"""
  return list(bestParams(lines).keys())[-1]

def bestParams(lines):
  """Return params produced best results"""
  return _searchForValue(lines, "BEST_PARAMS_")

def bestScores(lines):
  """Return best score for all model"""
  return _searchForValue(lines, "BEST_SCORE_")

def params(lines):
  """Return all params used for train all model"""
  return _searchForValue(lines, "Started GridSearchCV")

def _searchForValue(lines, v):
  m = {}
  for x in lines:
    if(v in x):
      c = x[x.index('(') + 1: x.index(')')]
      m[c] = x[x.index(': ') + 2:-2]
  
  return m

def logLines(log):
  f = open(log, "r")
  return [x for x in f]

def get_params(lines, label):
  line = bestParams(lines)[label]

  line_params = line.split(',')
  c_ = int(line_params[0][line_params[0].index(': ') + 2:])
  fuzzifier_ = line_params[1].split('.')[-1][:-2]

  k_ = line_params[2][6:line_params[2].index('(')]
  param_k_ = float(line_params[2][line_params[2].index('(')+1:line_params[2].index(')')])
  return (c_, fuzzifier_, (k_, param_k_))

# for c in range(40):
#   print(c, get_params(logLines(LOG_COMPELTE_PATH), str(c)))

params(logLines(LOG_COMPELTE_PATH))

0 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
1 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
2 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
3 (1, 'LinearFuzzifier', ('GaussianKernel', 0.25))
4 (1, 'QuantileLinearPiecewiseFuzzifier', ('GaussianKernel', 0.5))
5 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
6 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
7 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
8 (1000, 'QuantileLinearPiecewiseFuzzifier', ('GaussianKernel', 0.5))
9 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
10 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
11 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
12 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
13 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
14 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
15 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
16 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
17 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
18 (1, 'LinearFuzzifier', ('GaussianKernel', 0.5))
19 

{'0': "{'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>]",
 '1': "{'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>]",
 '10': "{'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>]",
 '11': "{'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>]",
 '12': "{'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLi

# Plot utils

In [0]:
def show_scatter_plot(data, labels):
  #plt.figure(figsize=(15,15))
  for lab in range(40):
          plt.scatter(data[labels==lab, 0],
                      data[labels==lab, 1],
                      label=lab,
                      c=[np.random.rand(3,)],
                      marker="$" + str(lab) + "$",
                      s=200
                      )
  plt.show()

def gr_membership_contour(estimated_membership, data, labels, label, with_label=False, precision=.25):
  plt.figure(figsize=(15, 15))
  data_filtered = [list(elem) for elem, l in list(zip(data, labels)) if l == label]

  offsetX = (maxX(data_filtered) - minX(data_filtered))/2
  offsetY = (maxY(data_filtered) - minY(data_filtered))/2

  x = np.arange(minX(data_filtered) - offsetX, maxX(data_filtered) + offsetX, precision)
  y = np.arange(minY(data_filtered) - offsetY, maxY(data_filtered) + offsetY, precision)
  X, Y = np.meshgrid(x, y)
  zs = np.array([estimated_membership([[x, y]])
                  for x,y in zip(np.ravel(X), np.ravel(Y))])
  Z = zs.reshape(X.shape)

  membership_contour = plt.contour(X, Y, Z,
                                    levels=(.1, .3, .5, .7, .8, .9, .95), colors='k')
  
  if(with_label):
    plt.clabel(membership_contour, inline=1)

  plt.title("Membership contour on test {}".format(label))
  show_scatter_plot(data, labels)

  plt.show()


def minX(x): return min(x, key=lambda t: t[0])[0]

def maxX(x): return max(x, key=lambda t: t[0])[0]

def minY(x): return min(x, key=lambda t: t[1])[1]

def maxY(x): return max(x, key=lambda t: t[1])[1]

# Estimations

In [0]:
models = {}

for c in get_classes(olivetti_labels):
  c_, fuzzifier_, (kernel_, param_) = get_params(logLines(LOG_COMPELTE_PATH), str(c))
  print(c_, str2fuzzifier(fuzzifier_), str2kernel(kernel_)(param_))
  print(map_labels(y_train, c))
  models[c] = fl.FuzzyInductor(c=c_, fuzzifier=str2fuzzifier(fuzzifier_), k=str2kernel(kernel_)(param_)).fit(X_train, map_labels(y_train, c))
  print(models[c].score(X_test, map_labels(y_test, c)))

In [0]:
def get_predictions(models, dataset):
  return [models[model].predict(dataset) for model in models]

predictionsOnTrain = get_predictions(models, X_train)
predictionsOnTest = get_predictions(models, X_test)

In [0]:
l_test = []
l_train = []

if(l == None):
  l = []
  for model in models:
    l.append(models[model].predict(X_test))

In [0]:
zipped_predictions = list(zip(*l))
results = list(zip([sorted(enumerate(predict), reverse=True, key=lambda x: x[1])[0][0] for predict in zipped_predictions], y_test))

print("Result: {:.4f}".format(sum([1 if elem[0] == elem[1] else 0 for elem in results])/len(y_test)))
print("Different result: {}".format([elem for elem in results if elem[0] != elem[1]]))

Result: 0.8333
Different result: [(39, 4), (0, 7), (0, 28), (0, 2), (0, 6), (0, 3), (7, 9), (2, 25), (0, 2), (14, 12), (0, 22), (0, 34), (29, 20), (22, 25), (0, 31), (0, 12), (0, 27), (0, 3), (8, 22), (24, 12)]


# Plotted data sample




In [0]:
show_scatter_plot(X_test, y_test)
show_scatter_plot(X_train, y_train)

In [0]:
model_26.score(X_test, [1 if elem == 26 else 0 for elem in y_test])

-0.029848089107371578

In [0]:
show_scatter_plot(olivetti_data_tsne, olivetti_labels)

In [0]:
for model in models:
  gr_membership_contour(models[model].predict, X_test, y_test, model)

Output hidden; open in https://colab.research.google.com to view.

# Model selection

---

Here you can find all of the training done with ``` grid_search ``` or ``` nested_grid_search ```



In [0]:
params = {
    'c': [1, 1000],
    'k': [fl.kernel.GaussianKernel(sigma=.5), fl.kernel.GaussianKernel(sigma=.25)],
    'fuzzifier': [fl.fuzzifiers.LinearFuzzifier, fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier]
}

logger = log_init()

In [0]:
for c in get_classes(olivetti_labels):
  if(c > int(lastAnalyzedClass(logLines(LOG_COMPELTE_PATH)))):
    grid_search(olivetti_data_tsne, olivetti_labels, params, logger=logger, y_needs_mapping=True, label=c)

In [0]:
lastAnalyzedClass(logLines(LOG_COMPELTE_PATH))

'39'

# AT&T dataset faces

In [0]:
import cv2

f = open(PROJ_COMPLETE_PATH + "AT&T_dataset_faces/dataset.txt", 'w')

for b in range(40):
  for a in range(10):
    data = cv2.imread(PROJ_COMPLETE_PATH + 'AT&T_dataset_faces/s' + str(b + 1) + '/' + str(a + 1) +'.pgm', -1)
    for line in data:
      for pixel in line:
        f.write(str(pixel) + " ")
    f.write('\n')

In [0]:
def atet_data():
  f_open = open(PROJ_COMPLETE_PATH + "AT&T_dataset_faces/dataset.txt", 'r')
  faces = []
  for line in f_open:
    faces.append([int(x) for x in line.split()])
  return np.asarray(faces)

def atet_labels():
  return np.ravel([[a] * 10 for a in range(40)])